In [8]:
import requests
import urllib.parse
def get_nhd_id(state, name):
    if name.lower() in ['none', 'unknown']:
        return None
    
    base_url = "https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query"
    params = {
        "where": f"gnis_name='{name}''",
        "outFields": "permanent_identifier,gnis_name,gnis_id",
        "f": "json"
    }
    query_url = f"{base_url}?{urllib.parse.urlencode(params)}"
    print(f"\nRequest URL for {state} - {name}:\n{query_url}")
    r = requests.get(base_url, params=params)
    data = r.json()
    if data.get("features"):
        f = data["features"][0]["attributes"]
        return {
            "name": name,
            "Permanent_ID": f.get("permanent_identifier"),
            "GNIS_ID": f.get("gnis_id")
        }
    else:
        return {"state": state, "name": name, "Permanent_ID": None, "GNIS_ID": None}

entries = [
    ("CA", "None"),
    ("CA", "Folsom Lake"),
    ("NV", "Pyramid Lake"),
    ("WA", "Brand New"),
    ("CA", "UNKNOWN"),
    ("NV", "Topaz Lake"),
    ("CA", "Walker Lake"),
    ("OR", "Pacific Ocean")
]

results = [get_nhd_id(state, name) for state, name in entries]
for r in results:
    print(r)



Request URL for CA - Folsom Lake:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Folsom+Lake%27%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&f=json

Request URL for NV - Pyramid Lake:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Pyramid+Lake%27%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&f=json

Request URL for WA - Brand New:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Brand+New%27%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&f=json

Request URL for NV - Topaz Lake:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Topaz+Lake%27%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&f=json

Request URL for CA - Walker Lake:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Walker+Lake%

In [10]:
import requests
import urllib.parse

def get_nhd_id(state, name):
    if name.lower() in ['none', 'unknown']:
        return None

    base_url = "https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query"
    params = {
        "where": f"gnis_name='{name}'",  # <-- fixed: removed extra quote
        "outFields": "permanent_identifier,gnis_name,gnis_id",
        "f": "json"
    }

    query_url = f"{base_url}?{urllib.parse.urlencode(params)}"
    print(f"\nRequest URL for {state} - {name}:\n{query_url}")

    r = requests.get(base_url, params=params)
    data = r.json()

    if data.get("features"):
        f = data["features"][0]["attributes"]
        return {
            "name": name,
            "Permanent_ID": f.get("permanent_identifier"),
            "GNIS_ID": f.get("gnis_id")
        }
    else:
        return {"state": state, "name": name, "Permanent_ID": None, "GNIS_ID": None}

entries = [
    ("CA", "None"),
    ("CA", "Folsom Lake"),
    ("NV", "Pyramid Lake"),
    ("WA", "Brand New"),
    ("CA", "UNKNOWN"),
    ("NV", "Topaz Lake"),
    ("CA", "Walker Lake"),
    ("OR", "Pacific Ocean")
]

results = [get_nhd_id(state, name) for state, name in entries]

for r in results:
    print(r)



Request URL for CA - Folsom Lake:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Folsom+Lake%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&f=json

Request URL for NV - Pyramid Lake:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Pyramid+Lake%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&f=json

Request URL for WA - Brand New:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Brand+New%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&f=json

Request URL for NV - Topaz Lake:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Topaz+Lake%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&f=json

Request URL for CA - Walker Lake:
https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name%3D%27Walker+Lake%27&outFields

In [ ]:
import pandas as pd

In [ ]:
import arcpy
import pandas as pd
import requests
import urllib.parse
import tempfile
import os

# ======================
# Configuration
# ======================
csv_path = r"source_waters.csv"
out_gdb = r"C:\GIS\Scratch.gdb"
out_fc = "NHD_Waterbodies_Combined"

# NHDPlus_HR MapServer (Waterbody layer)
BASE_URL = "https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query"

# ======================
# Functions
# ======================
def get_nhd_records(state, name):
    """Return *all* NHD features matching a given GNIS name."""
    if pd.isna(name) or str(name).lower() in ["none", "unknown"]:
        return [{
            "State": state, "Waterbody": name,
            "Permanent_ID": None, "GNIS_ID": None,
            "GNIS_Name": name, "geometry": None
        }]

    params = {
        "where": f"gnis_name = '{name}'",
        "outFields": "permanent_identifier,gnis_name,gnis_id",
        "returnGeometry": "true",
        "f": "json"
    }

    r = requests.get(BASE_URL, params=params)
    data = r.json()

    features = data.get("features", [])
    if not features:
        return [{
            "State": state, "Waterbody": name,
            "Permanent_ID": None, "GNIS_ID": None,
            "GNIS_Name": name, "geometry": None
        }]

    records = []
    for f in features:
        attrs = f["attributes"]
        geom = f["geometry"]
        records.append({
            "State": state,
            "Waterbody": name,
            "Permanent_ID": attrs.get("permanent_identifier"),
            "GNIS_ID": attrs.get("gnis_id"),
            "GNIS_Name": attrs.get("gnis_name"),
            "geometry": geom
        })
    return records


# ======================
# Main processing
# ======================
print("Reading input CSV...")
df = pd.read_csv(csv_path)

all_records = []
for _, row in df.iterrows():
    matches = get_nhd_records(row["State"], row["Waterbody"])
    for m in matches:
        # Combine CSV fields with NHD attributes
        m.update({
            "Count": row.get("Count"),
            "NAS_Infested": row.get("NAS_Infested"),
            "Manual_Override": row.get("Manual_Override")
        })
    all_records.extend(matches)

print(f"Total combined rows: {len(all_records)}")

# Convert to DataFrame
result_df = pd.DataFrame(all_records)

# ======================
# Convert JSON geometries to shapefiles via temp feature class
# ======================
temp_json = os.path.join(tempfile.gettempdir(), "nhd_temp.json")

# Filter out null geometries for those with no matches
matched = result_df[result_df["geometry"].notna()].copy()

if not matched.empty:
    # Write to JSON for ArcGIS conversion
    features = {
        "displayFieldName": "",
        "fieldAliases": {},
        "geometryType": "esriGeometryPolygon",
        "spatialReference": {"wkid": 4326},
        "fields": [],
        "features": [
            {"geometry": g, "attributes": {"idx": i}}
            for i, g in enumerate(matched["geometry"])
        ],
    }

    import json
    with open(temp_json, "w") as f:
        json.dump(features, f)

    arcpy.JSONToFeatures_conversion(temp_json, os.path.join(out_gdb, "temp_nhd"))
    temp_fc = os.path.join(out_gdb, "temp_nhd")

    # Add table attributes
    temp_sdf = arcpy.da.FeatureClassToNumPyArray(temp_fc, ["SHAPE@XY"])
else:
    temp_fc = None

# ======================
# Build final output table
# ======================
print("Writing final feature class...")

if temp_fc:
    # If we have geometries, convert result_df to Spatially Enabled DataFrame
    sdf = arcpy.da.NumPyArrayToTable(temp_sdf, os.path.join(out_gdb, "nhd_table"))
    # Add all attribute fields and join
    arcpy.management.AddField(temp_fc, "State", "TEXT")
    arcpy.management.AddField(temp_fc, "Waterbody", "TEXT")
    arcpy.management.AddField(temp_fc, "Permanent_ID", "TEXT")
    arcpy.management.AddField(temp_fc, "GNIS_ID", "TEXT")
    arcpy.management.AddField(temp_fc, "Count", "LONG")
    arcpy.management.AddField(temp_fc, "NAS_Infested", "TEXT")
    arcpy.management.AddField(temp_fc, "Manual_Override", "TEXT")

    # Could populate via update cursor here if needed.
    # (Simplified here for readability)

    arcpy.management.CopyFeatures(temp_fc, os.path.join(out_gdb, out_fc))
else:
    # No matches — create an empty feature class
    sr = arcpy.SpatialReference(4326)
    arcpy.management.CreateFeatureclass(out_gdb, out_fc, "POLYGON", spatial_reference=sr)
    print("No geometries found, empty feature class created.")

print(f"✅ Done. Output: {os.path.join(out_gdb, out_fc)}")


Reading input CSV...


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
import arcpy
import pandas as pd
import requests
import urllib.parse
import json
import os
from arcgis.features import GeoAccessor, GeoSeriesAccessor

# ======================
# CONFIG
# ======================
csv_path = r"source_waters.csv"
out_gdb = r"C:\GIS\Scratch.gdb"
out_fc = "NHD_Waterbodies_Combined"

BASE_URL = "https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query"

# ======================
# FUNCTION
# ======================
def get_nhd_records(state, name):
    """Query NHD for all features matching GNIS name."""
    if pd.isna(name) or str(name).lower() in ["none", "unknown"]:
        return [{
            "State": state,
            "Waterbody": name,
            "Permanent_ID": None,
            "GNIS_ID": None,
            "GNIS_Name": name,
            "geometry": None
        }]
    
    # Build query
    params = {
        "where": f"gnis_name = '{name}'",
        "outFields": "permanent_identifier,gnis_name,gnis_id",
        "returnGeometry": "true",
        "f": "json"
    }
    query_url = f"{BASE_URL}?{urllib.parse.urlencode(params)}"
    
    # Print for debug
    print(f"\n=== Querying {state} - {name} ===")
    print(f"URL: {query_url}")
    
    try:
        r = requests.get(BASE_URL, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Request failed for {name}: {e}")
        return [{
            "State": state, "Waterbody": name,
            "Permanent_ID": None, "GNIS_ID": None,
            "GNIS_Name": name, "geometry": None
        }]
    except json.JSONDecodeError:
        print(f"⚠️ Invalid JSON returned for {name}. Skipping.")
        return [{
            "State": state, "Waterbody": name,
            "Permanent_ID": None, "GNIS_ID": None,
            "GNIS_Name": name, "geometry": None
        }]

    features = data.get("features", [])
    if not features:
        print(f"ℹ️ No matches found for {name}")
        return [{
            "State": state, "Waterbody": name,
            "Permanent_ID": None, "GNIS_ID": None,
            "GNIS_Name": name, "geometry": None
        }]

    records = []
    for f in features:
        attrs = f["attributes"]
        geom = f.get("geometry")
        records.append({
            "State": state,
            "Waterbody": name,
            "Permanent_ID": attrs.get("permanent_identifier"),
            "GNIS_ID": attrs.get("gnis_id"),
            "GNIS_Name": attrs.get("gnis_name"),
            "geometry": geom
        })
    print(f"✅ Found {len(records)} feature(s) for {name}")
    return records


# ======================
# MAIN PROCESS
# ======================
print("Reading input CSV...")
df = pd.read_csv(csv_path)

all_records = []
for _, row in df.iterrows():
    matches = get_nhd_records(row["State"], row["Waterbody"])
    for m in matches:
        # Add CSV attributes
        m.update({
            "Count": row.get("Count"),
            "NAS_Infested": row.get("NAS_Infested"),
            "Manual_Override": row.get("Manual_Override")
        })
    all_records.extend(matches)

print(f"\nTotal records after expansion: {len(all_records)}")

result_df = pd.DataFrame(all_records)

# Convert geometry dicts to ArcGIS geometries for Spatially Enabled DataFrame
def arcgis_geom(g):
    if not g:
        return None
    try:
        return {"rings": g["rings"]} if "rings" in g else {"paths": g["paths"]}
    except Exception:
        return None

result_df["SHAPE"] = result_df["geometry"].apply(arcgis_geom)

# Convert to Spatially Enabled DataFrame
sdf = GeoAccessor.from_df(result_df, geometry_column="SHAPE", sr=4326)

# ======================
# SAVE OUTPUT
# ======================
out_path = os.path.join(out_gdb, out_fc)
print(f"\nWriting to {out_path} ...")
sdf.spatial.to_featureclass(location=out_path, sanitize_columns=True)
print("✅ Done! Spatial feature class created successfully.")


Reading input CSV...

=== Querying CA - Tenaya Lake - Yosemite NP ===
URL: https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name+%3D+%27Tenaya+Lake+-+Yosemite+NP%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&returnGeometry=true&f=json
ℹ️ No matches found for Tenaya Lake - Yosemite NP

=== Querying CA - Pacific - San Pedro Bay ===
URL: https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name+%3D+%27Pacific+-+San+Pedro+Bay%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&returnGeometry=true&f=json
ℹ️ No matches found for Pacific - San Pedro Bay

=== Querying CA - Sacramento River - Ord bend ===
URL: https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/9/query?where=gnis_name+%3D+%27Sacramento+River+-+Ord+bend%27&outFields=permanent_identifier%2Cgnis_name%2Cgnis_id&returnGeometry=true&f=json
ℹ️ No matches found for Sacramento River - Ord bend

=== Querying ID - Henrys Fork 

Exception: 'NoneType' object has no attribute 'projectAs'

In [2]:

nhd = get_fs_data('https://hydro.nationalmap.gov/arcgis/rest/services/nhd/MapServer/0')

c:\Users\amcclary\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hydro.nationalmap.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\amcclary\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hydro.nationalmap.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\amcclary\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hydro.nationalmap.gov'. Adding certificate verification is strongly advised. See: